In [7]:
import pandas as pd
import numpy as np

In [2]:
# Get latest S&P 500 list from wikipedia
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = table[0]

# Uncomment to save
# df.to_csv("S&P500.csv", columns=['Symbol', 'GICS Sector'])

In [30]:
res = {}
for index, row in df.iterrows():
    if row['GICS Sector'] not in res:
        res[row['GICS Sector']] = [row['Symbol']]
    else:
        res[row['GICS Sector']].append(row['Symbol'])


num_total_stocks = 0
num_days = 750
raw_sector_data = {}
for sector in res:
    sector_data = []
    num_sector_stocks = 0
    for ticker in res[sector]:
        try:
            data = pd.read_csv(f"Stocks/{ticker.lower()}.us.txt")
            stock_data = data.iloc[-1 * num_days:,]['Open'].to_numpy()
            
            if (stock_data.shape[0] == num_days):
                sector_data.append(stock_data)
                num_sector_stocks += 1
                num_total_stocks += 1
        except:
            pass
    sector_data = np.concatenate(sector_data, axis=0).reshape((num_sector_stocks, num_days))
    np.save(f"Data/train_and_val_{sector}", sector_data[0: int(num_days * 0.8)])
    np.save(f"Data/test_{sector}", sector_data[int(num_days * 0.8):,])

print(f"Total Stocks: {num_total_stocks}")

Total Stocks: 466
